In [ ]:
#by micheal sichilongo
#michealsichilongo@gmail.com
#youtube: @michealsichilongo

import serial
import cv2
import numpy as np
import tensorflow as tf
import time

# Define the serial port and baud rate
portVar = '/dev/ttyUSB0'  # Manually set the port to '/dev/ttyUSB0'
serialInst = serial.Serial()
serialInst.baudrate = 9600
serialInst.port = portVar

try:
    serialInst.open()
    print(f"Connected to {portVar}")
except serial.SerialException:
    print(f"Failed to connect to {portVar}. Check the port or connection.")
    exit()

def send_command_to_arduino(command):
    if command != '':
        serialInst.write(command.encode('utf-8'))  # Send the command to Arduino

def tflite_realtime_detection(modelpath, lblpath, min_conf=0.5):
    # Load the label map into memory
    with open(lblpath, 'r') as f:
        labels = [line.strip() for line in f.readlines()]

    # Load the TensorFlow Lite model into memory and specify the number of threads
    interpreter = tf.lite.Interpreter(model_path=modelpath)
    interpreter.allocate_tensors()

    # Get model details
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    height = input_details[0]['shape'][1]
    width = input_details[0]['shape'][2]

    # Open webcam
    cap = cv2.VideoCapture(0)

    last_person_time = time.time()
    lights_on = False

    while True:
        ret, frame = cap.read()

        if not ret:
            print("Error: Failed to capture frame")
            break

        # Resize frame to expected shape [1xHxWx3]
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        imH, imW, _ = frame.shape
        image_resized = cv2.resize(image_rgb, (width, height))

        # Normalize pixel values and convert data type to FLOAT32
        input_data = np.expand_dims(image_resized.astype(np.float32) / 255.0, axis=0)

        # Perform object detection by running the model with the frame as input
        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()

        # Retrieve detection results
        boxes = interpreter.get_tensor(output_details[1]['index'])[0]  # Bounding box coordinates of detected objects
        classes = interpreter.get_tensor(output_details[3]['index'])[0]  # Class index of detected objects
        scores = interpreter.get_tensor(output_details[0]['index'])[0]  # Confidence of detected objects

        person_detected = False

        for i in range(len(scores)):
            if ((scores[i] > min_conf) and (scores[i] <= 1.0)):
                detected_label = labels[int(classes[i])]
                if detected_label == 'person' and scores[i] > 0.7:
                    person_detected = True
                    last_person_time = time.time()
                    if not lights_on:
                        print('Lights ON')
                        send_command_to_arduino('ON')
                        lights_on = True

                # Draw rectangle and label on the frame
                xmin = int(max(1, (boxes[i][1] * imW)))
                ymin = int(max(1, (boxes[i][0] * imH)))
                xmax = int(min(imW, (boxes[i][3] * imW)))
                ymax = int(min(imH, (boxes[i][2] * imH)))
                cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (10, 255, 0), 2)
                object_name = labels[int(classes[i])]
                label = '%s: %d%%' % (object_name, int(scores[i] * 100))
                labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
                label_ymin = max(ymin, labelSize[1] + 10)
                cv2.rectangle(frame, (xmin, label_ymin - labelSize[1] - 10),
                              (xmin + labelSize[0], label_ymin + baseLine - 10), (255, 255, 255), cv2.FILLED)
                cv2.putText(frame, label, (xmin, label_ymin - 7), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)

        # Check if no person detected for 15 seconds
        if not person_detected and time.time() - last_person_time >= 6 and lights_on:
            print('Lights OFF')
            send_command_to_arduino('OFF')
            lights_on = False

        # Display the frame
        cv2.imshow('Smart Light', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to exit the loop
            break

    # Release video capture and close serial connection
    cap.release()
    serialInst.close()
    cv2.destroyAllWindows()

# Set up variables for running the user's model
PATH_TO_MODEL = 'your_model_path_here'   # Path to .tflite model file
PATH_TO_LABELS = 'your_model_label_path_here'   # Path to labelmap.txt file
min_conf_threshold = 0.5   # Confidence threshold

# Run real-time object detection!
tflite_realtime_detection(PATH_TO_MODEL, PATH_TO_LABELS, min_conf_threshold)
